In [13]:
import numpy as np
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [3]:
train_df = pd.read_csv(Path('2019loans.csv'))
test_df = pd.read_csv(Path('2020Q1loans.csv'))

In [14]:
# Convert categorical data to numeric and separate target feature for training data
x_train = pd.get_dummies(train_df.drop(columns=['target']))
y_train = train_df['target']

In [15]:
train_df.head(2)
x_train.head(2)


,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
0,7000.0,0.1894,256.38,75000.0,28.62,0.0,2.0,20.0,0.0,40414.0,...,0,1,1,0,1,0,1,0,1,0
1,40000.0,0.1614,975.71,102000.0,11.72,2.0,0.0,10.0,0.0,43531.0,...,0,1,0,1,1,0,1,0,1,0


In [16]:
y_test.head(2)

0    low_risk
1    low_risk
Name: target, dtype: object

In [17]:
# Convert categorical data to numeric and separate target feature for testing data
x_test = pd.get_dummies(test_df.drop(columns=['target']))
y_test = test_df['target']

In [18]:
y_test.head(2)

0    low_risk
1    low_risk
Name: target, dtype: object

In [19]:
x_test.head(2)

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N
0,40000.0,0.1033,856.40,128700.0,12.47,0.0,1.0,8.0,0.0,38113.0,...,1,0,1,0,1,1,0,0,1,1
1,24450.0,0.1430,572.72,44574.0,15.05,0.0,1.0,6.0,0.0,1665.0,...,0,0,1,0,1,1,0,1,0,1


In [ ]:
# I think that the random Forest will be more accurate on unscaled and less accurate on scaled data
#"Does random forest need input variables to be scaled or centered?" "No..."  
#"RF will tend to favour highly variable continuous predictors because there are more opportunities to partition the data."
# https://stats.stackexchange.com/questions/255765/does-random-forest-need-input-variables-to-be-scaled-or-centered

In [20]:
# adding missing dummy variables to testing set
for col in x_train.columns:
    if col not in x_test.columns:
        x_test[col] = 0
        

In [23]:
# Training the Logistic Regression model on the unscaled data and print the model score
classifier = LogisticRegression()
classifier.fit(x_train,y_train)
classifier.score(x_test,y_test)


0.5072309655465759

In [24]:
# Train a Random Forest Classifier model and print the model score
rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)
rfc.score(x_test,y_test)


0.6388770735857082

In [28]:
# Scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled =scaler.transform(x_train)
x_test_scaled =scaler.transform(x_test)

In [31]:
# Train the Logistic Regression model on the scaled data and print the model score
classifier = LogisticRegression()
classifier.fit(x_train_scaled,y_train)
classifier.score(x_test_scaled,y_test)
#so scaling the data is a must do for logistic regression

0.7598894087622289

In [32]:
# Train a Random Forest Classifier model on the scaled data and print the model score
rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)
rfc.score(x_test,y_test)
# we get slight increase in performance. So scaling the data didnt make it less accurate,
# but it didnt give us a comparable boost like scaled Logistic Regression. 


0.6452573373032752